In [0]:
# Databricks notebook: Exploración de datos OpenSky (EDA)

from pyspark.sql.functions import col, desc, round, countDistinct, avg

# 1. Cargar la tabla limpia desde Silver
df = spark.table("default.silver_opensky")

# 2. Vista previa general
print("👁 Vista general de los datos:")
display(df.limit(20))

# 3. Total de registros
print(f"📊 Total de registros: {df.count()}")

# 4. ¿Cuántos países distintos hay?
print("🌍 Países distintos:")
df.select("origin_country").distinct().show(truncate=False)

# 5. Número de vuelos por país
print("🛫 Número de vuelos por país:")
df.groupBy("origin_country").count().orderBy(desc("count")).show(10)

# 6. Altitud promedio por país
print("📈 Altitud promedio por país (geo_altitude):")
df.groupBy("origin_country").agg(round(avg("geo_altitude"), 2).alias("avg_altitude"))\
  .orderBy(desc("avg_altitude")).show(10)

# 7. Velocidad promedio por país
print("🚀 Velocidad promedio por país (velocity):")
df.groupBy("origin_country").agg(round(avg("velocity"), 2).alias("avg_velocity"))\
  .orderBy(desc("avg_velocity")).show(10)

# 8. ¿Cuántos vuelos están en tierra vs en el aire?
print("✈️ Estado del vuelo:")
df.groupBy("on_ground").count().show()

# 9. Distribución de altitud
print("📊 Distribución de geo_altitude (histograma aprox):")
display(df.select("geo_altitude"))

# 10. Datos nulos por columna
print("🔍 Conteo de valores nulos por columna:")
from pyspark.sql.functions import isnan, when, count

df.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df.columns
]).show()
